## ***Dimensionality Reduction***

#### The Curse of Dimensionality
* It is a fact that things change **Drastically** with increase in dimensions. For example , the chances of a random point to lie within 0.001 of a border in 2D is 0.4% which increase to 99.9999% in 10,000 dimension
* In theory the cure for curse of dimensionality is to increase the number of training instances to reach density , which is not very practical because for a data with just 100 feature will require equal number of instances as the number of atoms in observable universe.

#### The Main Approches for Dimensionality Reduction
***
##### **1) Projection**
* In most real world problem, training instances are *not* spread uniformly. Many Features are almost constant where as some are highly correlated. This tell us that the data can lie on much lower-dimension subspace of the high-dimensional space.
* The following is an example,see how training instances lie close to the plane. So to reduce dimension(3D-2D) we can take the projection of instances on the plane which will give us a 2D dataset
<br>
<img src='https://i.ibb.co/t4WwnB3/1.png' width=40% align=left><img src='https://i.ibb.co/MhrMrFg/2.png' width=30% align=middle><br>
* However , its not the best way to do dimensionality reduction as many(most) dataset have twist and turns for example Swis roll (dummy dataset) there is simply no way reduce dimension with projection as will create a more messy data than before<br>
<img src='https://i.ibb.co/sC88LQ0/3.png' width=35%><br>

##### 2) **Manifold Learning**
* The above mentioned Swiss roll data is an example of 2D manifold . That means it is 2D shape that is bent or rolled to a heigher dimension space(2D is rolled to 3D in Swiss roll). More generally a d-dimensional mainfold is a part of n-dimensional space and d < n.
* Many Dimensionality reduction work on modeling the manifold on which training instance lie called manifold learning. These algorithm relies on some manifold-assumptions called **manifold hypothesis** , which holds that most real world high-dimensional dataset lie closer to much lower-dimensional manifolds, another assumption is that our task(regression or classification) will be simpler if expresed in lower-dimensions
<img src='https://i.ibb.co/r5kGmsm/4.png' width=40%><br>
* we can see in above image that in first case our task(classification) become simpler after manifold reduction , where as it become complicated in second case (it was much simpler to classify in 3D with a x1=5 plane)

#### ***PCA*** (Principal Component Analysis)
The simple idea behind PCA is to find an optimal hyperplane and project the instances on it.
<br><br>***Finding right hyperplane to preserve maximum varience***
***
* Its resonable to select hyperplane which preserve maximum varience as it will most likely lose less information another way to justify , to find hyperplane which minimises the mean squared distance between the original point and projection<br><br>

***Principal Components Analysis***
***
* PCA identifies the axis that accounts largest ammount of varience in the training dataset. Number of these can be made on a n-dimensional space is n . and each axis is orthagonal to all the previous axis. 
* The unit vector of the iᵗʰ axis is called iᵗʰ *principal component* 
* We can find these *principal components* with help of *Singular Value Decomposition(SVD)* , which convert training set **X** into matrix multiplication of three matrix $UΣV^{T}$ where **V** contains all the principal components
* we can find PC with help of NumPy's scd() function which return 3 matrixs U,Σ,Vᵀ respectivly<br>
```python
import numpy as np
X_centered = X - X.mean(axis=0) #X is a 3D dataset
U,s,Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:,0]
c2 = Vt.T[:,1]
```
PCA assumes that the dataset is centered around the origin,sklearn's PCA class take care of it but data need to be centered otherwise
* Once we know all the PCs we can now reduce the dimensions to *d* dimensions (d<PCs), projection can simply be done by matrix multiplication of **X**(original dataset) and $W_{d}$ (it is a matrix with first d PCs{each column represent one PC})
```python
W2 = Vt.T[:,:2]
X2d = X_centered.dot(W2)
```
* **Sci-kit learn's application**
```python
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)
```
* Explained Variance Ratio : it is the ratio that tell us how much information/variance lies on that axis , it can accessed by explaiend_variance_ratio_ after fitting data in PCA class
```python
>>> pca.explaiend_variance_ratio_
array([0.84248607, 0.14631839])
```
this tell us that 84.2% of dataset's variance lies on $1^{st}$ PC and 14.6% on $2^{nd}$ and the remaining 1.2% on the third which we are not using
* **Choosing Right Number of Dimensions** : Instead of choosing a arbitary number as dimension we can choose the number of dimensions that add up to a sufficiently large portion or varience (eq 95%) . unless you reduciing dimension for data visualization(you just want 2-3 dimensions)<br>
to apply , insted or setting n_component to a number we can just set it to float between 0.0 and 1.0 which is the fraction of varience we want to cover
```python
pca = PCA(n_components=0.95) #for 95% of variance
X_reduces = pca.fit_transform(X)
```
* **PCA for Compression**: PCA can also be used for compression and Decompression , eg. We can reduce dimension of MNIST dataset to 95% variance and we can get back the image by .inverse_transform(reduces_matrix) , ofc. the decompressed image wont be same quality as it lost 5% varience but it would be comparable
```python
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)
```
<img src='https://i.ibb.co/HNLJxgp/Screenshot-from-2022-11-30-13-12-41.png' width=45%><br>
* **Randomized PCA** : if we set svd_solver='randomized' , SciKit-learn will use a stochastic algorithm called Randomized PCA, that is much faster than standard SVD with computational-complexity of $O(mXd^{2})+O(d^{3})$ insted of $O(mXn^{2})+O(n^{3})$
```python
rnd_pca = PCA(n_components=154,svd_solver='randomized')
X_red = rnd_pca.fit_transform(X)
```
by default it is set to 'auto' ,ie. SciKit-learn will use randomized SVD if m or n is greater than 500
<br><br>
* **Incremental PCA** : Many time we don't have enough resources(RAM) to load whole data at once but we want to perform PCA , in that case we can use IncrementalPCA class with small batches of data .
```python
from skllearn.decomposition import IncrementalPCA
inc_pca = IncrementalPCA(n_components=124)
for X_batch in np.array_split(X_train,100):
    inc_pca.partial_fit(X_batch)
X_red = inc_pca.transform(X_train)
```
alternative approch is to use NumPy's memmap() which only load the data that is needed , we can use the .fit() as memmap only bring the needed data by the operation
```python
X_mm = np.memmap(filename, dtype="float32", 
                   mode="readonly", shape=(m, n)) 
                   #m is n_instance and n is n_fet
batch_size = m // 100
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)
```
our data should already be converted to the memmap file (Memory-Mapped File)

#### ***kPCA*** (Kernel PCA)
* We know kernel trick is a mathematical technique that implicitly maps instances into very high dimensional space , hence creating it easier(possible) for classification and regression. As we used in SVM
* Since , some data don't work fine with linear dimensionality reduction technique so we have to first increase its dimension then apply PCA for better result
<br>

**Selecting Kernel and hyperparameter tuning**
***
* kPCA is an unsupervised algorithm(it need no target just X on which PCA will be applied) but its generally used as a preperation step for a supervised learning task(classification and regression),hence we can use grid search on it to find kernel and tune hyperparameters
```python
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf = Pipeline([
("kpca", KernelPCA(n_components=2)),
("log_reg", LogisticRegression())
])
param_grid = [{
"kpca__gamma": np.linspace(0.03, 0.05, 10),
"kpca__kernel": ["rbf", "sigmoid"]
}]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)
```

* Another totally unsupervised way is to choose kernel & hyperparameter with lowest reconstruction error
```python
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)
#### reconstructional error ####
from sklearn.metrics import mean_squared_error
err = mean_squared_error(X, X_preimage)
```

#### LLE (Locally Linear Embedding)
* it is also a nonlinear dimensionality reduction algorithm but unlike other which are based on projections , LLE is based on linear relationship of instance with its neighbour instances hence preserve the geomatric shape as well
*LLE works by first measuring how each training instance linearly relates to its closest neighbors (c.n.), and then looking for a low-dimensional representation of the training set where these local relationships are best preserved
```python
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)
```
Steps involved in LLE :-
* Step-1 : For each training instance $x^{(i)}$ algorithm will find its k closest neightbour , then it will find weights $w_{i,j}$(weight for $j^{th}$ neighbour of $i^{th}$ instance) such that the squared distance between $x^{(i)}$ and $Σ_{j=1}^{m}w_{(i,j)} x^{(j)}$ is minimum (W is matrix of all the weights $w_{i,j}$)
<img src='https://i.ibb.co/71mhgdM/6.png' width=45%><br>

* Step-2 : Now we will map the training instances into d-dimension(d<n) while keeping the local linear relation as intact as possible. If $z^{(i)}$ is the image of $x^{(i)}$ in the d-dimension then we have to minimise the squared distance between $z^{(i)}$ and $Σ_{j=1}^{m}w_{(i,j)} z^{(j)}$ . *it look very similar to the first step but now our goal is reversed in first step we were finding optimal weight keeping instances constant , and now we are keeping weight constant while finding best image ($z^{(i)}$) of the $x^{(i)}$ instance*
<img src='https://i.ibb.co/nzGbTpS/7.png' width=45%>

#### Other Dimensionality Reduciotion Techniques
***
* **Mulridimensional Scaling(MDS)** : Reduce Dimension while tying to preserve the distance between the instances
<br>

* **Isomap** : It creates a graph by connecting each instance to its nearest neighbors, then
reduces dimensionality while trying to preserve the geodesic distances9 between
the instances.
<br>

* **t-Distributed Stochastic Neighbor Embedding (t-SNE)**: reduces dimensionality while trying to keep similar instances close and dissimilar instances apart. It is mostly used for visualization, in particular to visualize clusters of instances in high-dimensional space (e.g., to visualize the MNIST images in 2D)
<br>

* **Linear Discriminant Analysis (LDA)**: is actually a classification algorithm, but during training it learns the most discriminative axes between the classes, and these axes can then be used to define a hyperplane onto which to project the data. The benefit is that the projection will keep classes as far apart as possible, so LDA is a good technique to reduce dimensionality before running another classification algorithm such as an SVM classifier.